In [1]:
import arcpy
from arcpy import env
import os
from arcgis import GIS
from arcgis.features import GeoAccessor
import pandas as pd
import numpy as np
from zipfile import ZipFile

arcpy.env.overwriteOutput = True
arcpy.env.parallelProcessingFactor = "90%"

# show all columns
pd.options.display.max_columns = None

arcpy.env.overwriteOutput = True
arcpy.CheckOutExtension("Spatial")

# pd.DataFrame.spatial.from_featureclass(???)
# df.spatial.to_featureclass(location=???,sanitize_columns=False)

'CheckedOut'

In [2]:
if not os.path.exists('Outputs'):
    os.makedirs('Outputs')
    
outputs = ['.\\Outputs', "mmn_scratch.gdb", 'mmn_results.gdb']
gdb = os.path.join(outputs[0], outputs[1])
gdb2 = os.path.join(outputs[0], outputs[2])

if not arcpy.Exists(gdb):
    arcpy.CreateFileGDB_management(outputs[0], outputs[1])

if not arcpy.Exists(gdb2):
    arcpy.CreateFileGDB_management(outputs[0], outputs[2])

## Create empty hdf

In [3]:
# store path for new hdf
hdf_name = 'Multi_Modal_Network_20220428'
new_hdf = os.path.join('.\\Outputs', hdf_name + '.h5')
print(new_hdf)

# if the h5 exists already delete it; it will not overwrite
if os.path.exists(new_hdf):
    try:
        new_hdf.close()
    except:
        pass
    
    os.remove(new_hdf)

# Create empty h5   
hdf = pd.HDFStore(new_hdf)   

.\Outputs\Multi_Modal_Network_20220428.h5


## Inspect original network tables

In [4]:
osm = r".\Inputs\osm_wfrc.h5"
store = pd.HDFStore(osm)
tables = list(store.keys())
print(tables)

['/edges', '/nodes']


In [5]:
store['edges'].head(5)

,from,to,weight
31354714,25323002,25323003,58.672306
51904539,40243023,40243025,292.575745
51904540,40243025,40243028,331.527008
51904541,40243028,6240765,127.711517
51904542,40243024,40243025,123.560577


In [6]:
#nodes
store['nodes'].head(5)

,x,y
35651665,1540812.750,7442087.5
35651666,1540787.500,7440621.5
35651667,1540787.125,7440260.5
35651668,1540772.250,7439106.0
35651669,1531160.500,7419242.5


## Load Multimodal Network

In [7]:
taz900 = r".\Inputs\TAZ_900_ID_Only.shp"
reporting_areas = r".\Inputs\REMM_Reporting_Areas.shp"
mmn = r".\Inputs\MM_NetworkDataset_04072022.gdb"
network = os.path.join(mmn, 'NetworkDataset\\BikePedAuto')
network_lyr = arcpy.MakeFeatureLayer_management(network,"network_lyr")

## Subset network using TAZ boundary and Cartocode Attribute

In [8]:
#filter using modeling area
arcpy.SelectLayerByLocation_management(network_lyr, 'INTERSECT', reporting_areas, selection_type='NEW_SELECTION')

# filter based on cartocode
query = """ CartoCode NOT IN ('1 Interstates','10 Other Federal Aid Eligible Local Roads','7 Ramps, Collectors','13 Non-road feature','14 Driveway','15 Proposed') """
arcpy.SelectLayerByAttribute_management(network_lyr, 'SUBSET_SELECTION', query)

# copy the line features
filtered_lines = arcpy.FeatureClassToFeatureClass_conversion(network_lyr, gdb, 'filtered_lines')

# Add unique ID
unique_id_field = 'id'
arcpy.AddField_management(filtered_lines, field_name=unique_id_field, field_type='LONG')
arcpy.CalculateField_management(filtered_lines, unique_id_field, '"!OBJECTID!"')

<Result '.\\Outputs\\mmn_scratch.gdb\\filtered_lines'>

# Start pts

In [9]:
print('--generating start points')
start_pts = arcpy.FeatureVerticesToPoints_management(filtered_lines, os.path.join(gdb, 'start_pts_initial'), 'START')

# Delete extra Start nodes, accounts for rare instance of multiple end nodes created
print('--checking for extra start points')
arcpy.AddField_management(start_pts, field_name='Temp_ID', field_type='LONG')
arcpy.CalculateField_management(start_pts, 'Temp_ID', '"!OBJECTID!"')
start_pts_sorted = arcpy.Sort_management(start_pts, os.path.join(gdb, 'start_pts'), sort_field=[["Temp_ID", "ASCENDING"]])

previous_id = None
with arcpy.da.UpdateCursor(start_pts_sorted, ['temp_id']) as cursor:
    for row in cursor:
        
        if row[0] == previous_id:
            #print("--Current:{}, Previous:{}".format(row[0], previous_id))
            cursor.deleteRow()
        previous_id = row[0]

arcpy.DeleteField_management(start_pts, 'Temp_ID')
start_pts = start_pts_sorted

# add xy coords to start points
arcpy.AddField_management(start_pts, field_name="xcoord", field_type='double')
arcpy.AddField_management(start_pts, field_name="ycoord", field_type='double')

# this might allow this section to run in command line
with arcpy.da.UpdateCursor(start_pts, ['xcoord', 'ycoord', 'SHAPE@X', 'SHAPE@Y']) as cursor:
    for row in cursor:
        row[0] = row[2]
        row[1] = row[3]
        cursor.updateRow(row)
        
# create xy key to start points
arcpy.AddField_management(start_pts, field_name="XY_Key", field_type='string')
arcpy.CalculateField_management(start_pts,"XY_Key",'"!{}!|!{}!"'.format('xcoord', 'ycoord'))

--generating start points
--checking for extra start points


<Result '.\\Outputs\\mmn_scratch.gdb\\start_pts'>

## End points

In [10]:
print('--generating end points')

# get end points for each line
end_pts = arcpy.FeatureVerticesToPoints_management(filtered_lines, os.path.join(gdb, 'end_pts_initial'), 'END')

# Delete extra End nodes, accounts for rare instance of multiple end nodes created
print('--checking for extra end points')
arcpy.AddField_management(end_pts, field_name='Temp_ID', field_type='LONG')
arcpy.CalculateField_management(end_pts, 'Temp_ID', '"!OBJECTID!"')
end_pts_sorted = arcpy.Sort_management(end_pts, os.path.join(gdb, 'end_pts'), sort_field=[["Temp_ID", "DESCENDING"]])

previous_id = None
with arcpy.da.UpdateCursor(end_pts_sorted, ['temp_id']) as cursor:
    for row in cursor:
        
        if row[0] == previous_id:
            #print("--Current:{}, Previous:{}".format(row[0], previous_id))
            cursor.deleteRow()
        previous_id = row[0]

arcpy.DeleteField_management(end_pts, 'Temp_ID')
end_pts = end_pts_sorted

# add xy coords to end points
arcpy.AddField_management(end_pts, field_name="xcoord", field_type='double')
arcpy.AddField_management(end_pts, field_name="ycoord", field_type='double')

# this might allow this section to run in command line
with arcpy.da.UpdateCursor(end_pts, ['xcoord', 'ycoord', 'SHAPE@X', 'SHAPE@Y']) as cursor:
    for row in cursor:
        row[0] = row[2]
        row[1] = row[3]
        cursor.updateRow(row)

# add xy key to end points
arcpy.AddField_management(end_pts, field_name="XY_Key", field_type='string')
arcpy.CalculateField_management(end_pts,"XY_Key",'"!{}!|!{}!"'.format('xcoord', 'ycoord'))

--generating end points
--checking for extra end points


<Result '.\\Outputs\\mmn_scratch.gdb\\end_pts'>

# Join keys from lines and merge start and end points together

In [11]:
filtered_lines_lyr = arcpy.MakeFeatureLayer_management(filtered_lines,"filtered_lines")

# join with centerlines, copy xy key
arcpy.AddJoin_management(filtered_lines_lyr, unique_id_field, start_pts, unique_id_field)
arcpy.CalculateField_management(filtered_lines_lyr,"Start_Key",'!{}!'.format('start_pts.XY_Key'))
arcpy.RemoveJoin_management (filtered_lines_lyr)

# join with centerlines, copy xy key
arcpy.AddJoin_management(filtered_lines_lyr, unique_id_field, end_pts, unique_id_field)
arcpy.CalculateField_management(filtered_lines_lyr,"End_Key",'!{}!'.format('end_pts.XY_Key'))
arcpy.RemoveJoin_management (filtered_lines_lyr)

# Create 'both ends' nodes data set by merging start nodes and end nodes
print('--merging start and end points')    
merged_pts = arcpy.Merge_management([start_pts, end_pts], os.path.join(gdb, "merged_pts"))

# Remove duplicate nodes
print('--deleting duplicate nodes')
arcpy.DeleteIdentical_management(merged_pts, "XY_Key")

--merging start and end points
--deleting duplicate nodes


<Result '.\\Outputs\\mmn_scratch.gdb\\merged_pts'>

## Create links

In [12]:
print('Creating Links') 

# calculate distance in meters
arcpy.AddField_management(filtered_lines, field_name="weight", field_type='float')
with arcpy.da.UpdateCursor(filtered_lines, ['weight', 'SHAPE@LENGTH']) as cursor:
    for row in cursor:
        row[0] = row[1]
        cursor.updateRow(row)    

Creating Links


## Table formatting and export

In [13]:
print('Creating final outputs')
nodes_dataframe  = pd.DataFrame.spatial.from_featureclass(merged_pts[0])
links_dataframe  = pd.DataFrame.spatial.from_featureclass(filtered_lines[0])

print('--formatting nodes')
nodes_dataframe_formatted = nodes_dataframe[['OBJECTID', 'xcoord', 'ycoord', 'XY_Key','SHAPE']].copy()
# nodes_dataframe_formatted = nodes_dataframe_formatted.rename(columns={"OBJECTID": "node_id"})
# nodes_dataframe_formatted = nodes_dataframe_formatted.sort_values(by=['node_id'])
nodes_dataframe_formatted.columns = ['node_id', 'x', 'y','XY_Key','SHAPE']
nodes_dataframe_formatted = nodes_dataframe_formatted.sort_values(by=['node_id'])

print('--exporting nodes to shape, csv, and h5')
nodes_dataframe_formatted.spatial.to_featureclass(location=os.path.join(gdb2, 'nodes'),sanitize_columns=False)

del nodes_dataframe_formatted['SHAPE']
nodes_dataframe_formatted[['node_id', 'x', 'y']].to_csv(r'.\Outputs\nodes.csv',index=False)
hdf.put('nodes', nodes_dataframe_formatted[['node_id', 'x', 'y']].set_index('node_id'), format='t', data_columns=True)

Creating final outputs
--formatting nodes
--exporting nodes to shape, csv, and h5


In [14]:
links_dataframe.columns

Index(['OBJECTID', 'Name', 'Oneway', 'Speed', 'AutoNetwork', 'BikeNetwork',
       'PedNetwork', 'SourceData', 'DriveTime', 'BikeTime', 'PedestrianTime',
       'Length_Miles', 'ConnectorNetwork', 'CartoCode', 'AADT', 'AADT_YR',
       'BIKE_L', 'BIKE_R', 'VERT_LEVEL', 'id', 'Start_Key', 'End_Key',
       'weight', 'SHAPE'],
      dtype='object')

In [15]:
print('--formatting links')

# Create from/to node columns
links_dataframe = links_dataframe.assign(from_node='', to_node='', from_x='', from_y='', to_x='', to_y='')

# Join with nodes to get start node IDs
links_dataframe_temp = links_dataframe.merge(nodes_dataframe_formatted, left_on = 'Start_Key', right_on ='XY_Key' , how='inner')
links_dataframe_temp['from_node'] = links_dataframe_temp['node_id']
links_dataframe_temp['from_x'] = links_dataframe_temp['x']
links_dataframe_temp['from_y'] = links_dataframe_temp['y']

# Join with nodes to get end node IDs
links_dataframe_temp = links_dataframe_temp.merge(nodes_dataframe_formatted, left_on = 'End_Key', right_on='XY_Key' , how='inner')
links_dataframe_temp['to_node'] = links_dataframe_temp['node_id_y']
links_dataframe_temp['to_x'] = links_dataframe_temp['x_y']
links_dataframe_temp['to_y'] = links_dataframe_temp['y_y']

# subset and rename columns
links_dataframe_formatted = links_dataframe_temp[['OBJECTID', 'from_node', 'to_node', 'weight','from_x', 'from_y', 'to_x', 'to_y', 'SHAPE']].copy()
links_dataframe_formatted = links_dataframe_formatted.rename(columns={"OBJECTID": "link_id"})
links_dataframe_formatted = links_dataframe_formatted.sort_values(by=['link_id'])
links_field_names = ['edge_id',  'from','to', 'weight','from_x', 'from_y', 'to_x', 'to_y','SHAPE']
links_dataframe_formatted.columns = links_field_names
links_dataframe_formatted['weight'] = links_dataframe_formatted['weight'].round(2)

print('--exporting links to shape, csv, and h5')
links_dataframe_formatted.spatial.to_featureclass(location=os.path.join(gdb2, 'edges'),sanitize_columns=False)

# export to csv and h5
del links_dataframe_formatted['SHAPE']
del links_dataframe_formatted['from_x']
del links_dataframe_formatted['from_y']
del links_dataframe_formatted['to_x']
del links_dataframe_formatted['to_y']
links_dataframe_formatted.to_csv(r'.\Outputs\edges.csv',index=False)
hdf.put('edges', links_dataframe_formatted.set_index('edge_id'), format='t', data_columns=True)

--formatting links
--exporting links to shape, csv, and h5


C:\Users\jreynolds\AppData\Local\ESRI\conda\envs\arcgispro-py3-remm\lib\site-packages\tables\path.py:155: NaturalNameWarning: object name is a Python keyword: 'from'; you will not be able to use natural naming to access this object; using ``getattr()`` will still work, though
  check_attribute_name(name)


In [16]:
# close the h5
hdf.close()

In [17]:
# zip it up for distribution
ZipFile(os.path.join('.\\Outputs', hdf_name + '.zip'), mode='w').write(new_hdf, arcname=hdf_name + '.h5')